In [97]:
import pandas as pd
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import json
import time
# from lemmatize import lemmatize
x_chat = chatGPT()
x_gemini = gemini()




In [14]:
sample_df = pd.read_csv('../output/results v3/df_with_examples.csv')
job_cols = [[f"Job{i}",f"Job{i}_title"] for i in range(1,4)]
# for job in job_cols:
    # for col in job:
        # sample_df[col+"_lemmatize"] = sample_df[col].apply(lambda x: lemmatize(x))
# 
# lemmatize_cols = [[f"Job{i}_lemmatize",f"Job{i}_title_lemmatize"] for i in range(1,4)]

embedding_cols = []
for i in range(1,4):  
    embedding_cols += [f"Job{i}_embedding",f"Job{i}_title_embedding"]

results = []
for i, x in list(sample_df.iterrows()):
    name = x["organization name"]
    print(f"Processing {str(i), len(results)}: {name}")
    job_embeddings = [name]
    for job in job_cols: 
        for col in job:
            if pd.isnull(x[col]):
                job_embeddings.append(np.nan)
                continue
            job_embeddings.append(x_chat.get_embedding(x[col]))
    results.append(job_embeddings)


results_df = pd.DataFrame(results, columns=['organization name']+embedding_cols)
sample_df = sample_df.merge(results_df, on="organization name")

sample_df.to_csv('../output/results v3/df_with_examples_embeddings.csv', index=False)

Processing ('0', 0): Pika
Processing ('1', 1): Contextual AI
Processing ('2', 2): Sierra
Processing ('3', 3): Essential AI
Processing ('4', 4): Liquid AI
Processing ('5', 5): Lindy
Processing ('6', 6): LlamaIndex
Processing ('7', 7): MagicSchool AI
Processing ('8', 8): Norm AI
Processing ('9', 9): Patronus AI
Processing ('10', 10): Sixfold AI
Processing ('11', 11): Voltage Park
Processing ('12', 12): Elicit
Processing ('13', 13): Mercor
Processing ('14', 14): Guardrails AI
Processing ('15', 15): VectorShift
Processing ('16', 16): Portkey AI
Processing ('17', 17): MyShell
Processing ('18', 18): Genmo
Processing ('19', 19): Braintrust Data
Processing ('20', 20): Paxton AI
Processing ('21', 21): Oliv AI
Processing ('22', 22): Mindtrip
Processing ('23', 23): Worth AI
Processing ('24', 24): Kintsugi AI
Processing ('25', 25): Greenlite
Processing ('26', 26): Arkifi
Processing ('27', 27): Retell AI
Processing ('28', 28): LastMile AI
Processing ('29', 29): Rhythms
Processing ('30', 30): Stack 

In [8]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings_v1.csv')
# df_exp = df_exp.rename({"Embeddings":"task_embedding"},axis=1)
# df_exp.to_csv('../output/onet/gpt_exposure_embeddings_v1.csv')

In [9]:
df_exp.task_embedding = df_exp.task_embedding.apply(lambda x: x.strip("[]").split(", "))

In [10]:
task_embeddings = []
for x in df_exp.task_embedding:
    task_embeddings.append([float(y) for y in x])
df_exp.task_embedding = task_embeddings


task_embeddings = np.array(task_embeddings)
task_embeddings = np.vstack(task_embeddings)
task_embeddings.shape

(11582, 3072)

In [11]:
df_exp.title_embedding = df_exp.title_embedding.apply(lambda x: x.strip("[]").split(", "))
title_embeddings = []
for x in df_exp.task_embedding:
    title_embeddings.append([float(y) for y in x])
df_exp.title_embedding = title_embeddings


In [4]:
sample_df = pd.read_csv('../output/results v3/df_with_examples_embeddings.csv')
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

# list(sample_df.Job1_embedding.apply(ast.literal_eval))
for i in range(3):
    sample_df[f"Job{i+1}_embedding"] = sample_df[f"Job{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
    sample_df[f"Job{i+1}_title_embedding"] = sample_df[f"Job{i+1}_title_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])




In [12]:

results = []
# results = list(pd.read_csv("../output/results v3/onet_df.csv").values)

for index, row in list(sample_df.iterrows())[2031:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(1): 
        example = row[f"Example{i+1}"]
        job = row[f'Job{i+1}']
        job_title = row[f'Job{i+1}_title']
        print(f"\nProcessing Example: {example} \nJob: {job}\n")

        job_embedding = np.array(row[f'Job{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Job{i+1}_title_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, task_embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['Title', 'Task', 'description','title_embedding']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            onet_title_embedding = np.array([onet[3]])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[0]}: {onet[1]} {onet[2]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 5 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
        onet_df.to_csv("../output/results v3/onet_df.csv", index=False)
        
    print("***********************\n\n")


***********************
Processing 2031
toby
Toby is a SaaS platform that utilizes AI to provide real-time translation for both speech and text, enabling seamless communication and collaboration with individuals across language barriers. This technology streamlines global workflows and fosters inclusive environments.

Processing Example: A customer service representative uses toby to translate customer inquiries in real-time, enabling them to provide efficient and accurate support to a global customer base. 
Job: Customer Service Representatives that respond to customer inquiries regarding orders, products, services, or company information.

Customer Service Representatives: Confer with customers by telephone or in person to provide information about products or services, take or enter orders, cancel accounts, or obtain details of complaints. Customer service representatives that confer with customers by telephone or in person to provide information about products or services, take or 

In [13]:
onet_df.to_csv("../output/results v3/onet_df.csv", index=False)

In [7]:
# pd.read_csv("../output/results v3/onet_df.csv")


sample_df[sample_df["organization name"]=="Lake Effect Technology Solutions"]



,organization name,full_description,industries,headquarters location,founded date,description,CB rank,headquarters region,postal code,estimated revenue,...,situation3_conf_interval_reasoning,Job1_embedding,Job1_title_embedding,Job2_embedding,Job2_title_embedding,Job3_embedding,Job3_title_embedding,Job1_ONET,Job2_ONET,Job3_ONET
2030,Lake Effect Technology Solutions,Technology consulting with an emphasis on cutt...,"Consulting, Information Technology, Software","Holmen, Wisconsin, United States",2024-01-24,Technology Consulting,—,"Great Lakes, Midwestern US",—,—,...,This example falls within the broader scope of...,"[-0.03472146764397621, -0.029757382348179817, ...","[-0.018058864399790764, -0.010909165255725384,...","[-0.004524268209934235, -0.033689260482788086,...","[-0.00014851520245429128, -0.02311815693974495...","[-0.01093626394867897, 0.0009015528485178947, ...","[-0.01748700998723507, 0.017996251583099365, -...",N/A,N/A,N/A


In [7]:
onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
onet_df.to_csv("../output/onet_df.csv", index=False)

NameError: name 'onet_df' is not defined

In [10]:
s1 = "Animators and Multimedia Artists that create storyboards to illustrate scenes or action sequences for films, television programs, or other media."
s2 = "Special effects artists and animators that apply story development, directing, cinematography, and editing to animation to create storyboards that show the flow of the animation and map out key scenes and characters."



In [89]:
row[f'Job{i+1}_title_lemmatize']

'Computer Information Systems Managers'

In [12]:
s1 = lemmatize(s1)
s2 = lemmatize(s2)

In [80]:
s2

"Computer Information Systems Managers evaluate cloud service offering different provider select suitable option company 's need"

In [13]:
e1 = x_chat.get_embedding(s1)
#e1 = df_exp[df_exp.description.str.contains("managers that evaluate the organization")].task_embedding.iloc[0]
e2 = x_chat.get_embedding(s2)

#e2 = x_chat.get_embedding("Computer Information Systems Managers evaluate cloud service offering different provider select suitable option company 's need")
e1 = np.array(e1).reshape(1,-1)
e2 = np.array(e2).reshape(1,-1)
cosine_similarity(e1,e2, 'cosine')[0][0]

0.6625274673336069

In [31]:
df_exp.head()

NameError: name 'df_exp' is not defined

,organization_name,example,situation_conf_interval,situation_conf_interval_reasoning,job,onet_title,onet_task,example_job_title,task_similarity,job_title_similarity
0,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,"Camera Operators, Television, Video, and Film","Edit video for broadcast productions, includin...",Video Editors,0.734247,0.631970
1,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,Film and Video Editors,"Set up and operate computer editing systems, e...",Video Editors,0.702733,0.615354
2,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,Film and Video Editors,Program computerized graphic effects.,Video Editors,0.681118,0.652431
3,Contextual AI,A marketing manager uses Contextual AI to gene...,8.0,This example uses the product to generate cont...,"Marketing Specialists that write, edit, and pr...",Search Marketing Strategists,Create content strategies for digital media.,Marketing Specialists,0.640236,0.585241
4,Contextual AI,A marketing manager uses Contextual AI to gene...,8.0,This example uses the product to generate cont...,"Marketing Specialists that write, edit, and pr...",Public Relations Specialists,Write press releases or other media communicat...,Marketing Specialists,0.607678,0.482784
...,...,...,...,...,...,...,...,...,...,...
5893,Vanguard AI,A marketing team at a small business uses Vang...,8.0,This example directly relates to a common task...,Marketing Specialists that develop and impleme...,Search Marketing Strategists,Participate in the development or implementati...,Marketing Specialists,0.651843,0.594948
5894,Vanguard AI,A marketing team at a small business uses Vang...,8.0,This example directly relates to a common task...,Marketing Specialists that develop and impleme...,Spa Managers,Develop or implement marketing strategies.,Marketing Specialists,0.644694,0.524561
5895,WealthSmyth,A financial advisor uses WealthSmyth to analyz...,9.0,This example focuses on the AI-powered analysi...,"Securities, Commodities, and Financial Service...","Securities, Commodities, and Financial Service...",Track and analyze factors that affect price mo...,"Securities, Commodities, and Financial Service...",0.815297,0.684818
5896,WealthSmyth,A financial advisor uses WealthSmyth to analyz...,9.0,This example focuses on the AI-powered analysi...,"Securities, Commodities, and Financial Service...","Securities, Commodities, and Financial Service...",Monitor markets or positions.,"Securities, Commodities, and Financial Service...",0.791539,0.713737


In [17]:
title_embeddings = np.array(df_exp.title_embedding)
title_embeddings = np.vstack(title_embeddings)

In [147]:
onet_df = pd.read_csv("../output/results v3/onet_df.csv")
onet_df = onet_df[[col for col in onet_df.columns if "titles" not in col]]
onet_df[["Job_similarity_titles"]]= 'N/A'
len(onet_df)

6093

In [ ]:


for index, row in list(sample_df.iterrows())[0:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    for i in range(3): 
        example = row[f"Example{i+1}"]
        job_title = row[f'Job{i+1}_title']
        print(f"\n\nJob Title: {job_title}\n")

       
        job_title_embedding = np.array(row[f'Job{i+1}_title_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_title_embedding, title_embeddings, 'cosine')
        #get all cosine sim that have higher than 70 percent similarity
        high_sim_indices = np.where(cosine_sim >= 0.5)[1]
        high_sim_values = cosine_sim[0][high_sim_indices]
        
        titles = df_exp.iloc[high_sim_indices][['Title','O*NET-SOC Code']].values
        # titles = np.unique(list(titles))
        
        sorted_similarities = sorted(zip(high_sim_values, titles), reverse=True)
        sorted_titles = [(x,list(data)) for x, data in sorted_similarities]
        # sorted_titles = np.vstack(sorted_titles)
        
        onet_df.loc[index*3+i, f"Job_similarity_titles"] = json.dumps(sorted_titles)
       

    if index % 100 == 0:
        print("**Saving Results**\n\n")
        onet_df.to_csv("../output/results v3/onet_df.csv", index=False)
        time.sleep(1)
        
    print("***********************\n\n")

***********************
Processing 0
Pika
Pika is an AI-powered platform that transforms captions and still images into professional-looking videos. With its intuitive interface, Pika simplifies video creation and editing, making it accessible to users of all skill levels.


Job Title: Video Editors



Job Title: Multimedia Artists and Animators



Job Title: Graphic Designers

**Saving Results**


***********************


***********************
Processing 1
Contextual AI
Contextual AI provides a cutting-edge generative AI platform designed specifically for business applications. Their technology empowers employees to generate creative content, automate workflows, and gain insights from data, all while ensuring alignment with company-specific context and knowledge.


Job Title: Marketing Specialists



Job Title: Human Resources Specialists



Job Title: Sales Representatives

***********************


***********************
Processing 2
Sierra
Sierra is an AI company that develops 

In [ ]:
titles = list([x[0]+" , "+x[1]for x in titles])

In [88]:
print("here")

here


In [128]:
onet_df.to_csv("../output/results v3/onet_df.csv", index=False)

In [146]:
onet_df.head(10)

,organization_name,example,situation_conf_interval,situation_conf_interval_reasoning,job,onet_title,onet_task,example_job_title,task_similarity,job_title_similarity,Job_similarity_titles
0,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,"Camera Operators, Television, Video, and Film","Edit video for broadcast productions, includin...",Video Editors,0.734247,0.631970,"[[0.6524310909942744, [""Film and Video Editors..."
1,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,Film and Video Editors,"Set up and operate computer editing systems, e...",Video Editors,0.702733,0.615354,"[[0.6118800418773336, [""Special Effects Artist..."
2,Pika,A social media manager uses Pika to quickly ge...,8.0,This task specifically involves editing and cr...,Video Editors that create or edit video produc...,Film and Video Editors,Program computerized graphic effects.,Video Editors,0.681118,0.652431,"[[0.6726403093293662, [""Graphic Designers"", ""2..."
3,Contextual AI,A marketing manager uses Contextual AI to gene...,8.0,This example uses the product to generate cont...,"Marketing Specialists that write, edit, and pr...",Search Marketing Strategists,Create content strategies for digital media.,Marketing Specialists,0.640236,0.585241,"[[0.6210324475838761, [""Search Marketing Strat..."
4,Contextual AI,A marketing manager uses Contextual AI to gene...,8.0,This example uses the product to generate cont...,"Marketing Specialists that write, edit, and pr...",Public Relations Specialists,Write press releases or other media communicat...,Marketing Specialists,0.607678,0.482784,"[[0.7507350812022153, [""Human Resources Specia..."
5,Contextual AI,A marketing manager uses Contextual AI to gene...,8.0,This example uses the product to generate cont...,"Marketing Specialists that write, edit, and pr...",Proofreaders and Copy Markers,"Write original content, such as headlines, cut...",Marketing Specialists,0.606426,0.331235,"[[0.671704798965495, [""Sales Representatives, ..."
6,Sierra,A customer service representative uses Sierra'...,9.0,This is a common task for customer service rep...,Customer Service Representatives that answer q...,Customer Service Representatives,Confer with customers by telephone or in perso...,Customer Service Representatives,0.724976,0.712624,N/A
7,Sierra,A customer service representative uses Sierra'...,9.0,This is a common task for customer service rep...,Customer Service Representatives that answer q...,"Sales Representatives of Services, Except Adve...","Answer customers' questions about services, pr...",Customer Service Representatives,0.719068,0.662352,N/A
8,Sierra,A customer service representative uses Sierra'...,9.0,This is a common task for customer service rep...,Customer Service Representatives that answer q...,Telemarketers,"Explain products or services and prices, and a...",Customer Service Representatives,0.663429,0.561342,N/A
9,Essential AI,A marketing team uses Essential AI to generate...,8.0,"While not specifically for PR, this task overl...","Public Relations Specialists that write, edit,...",Public Relations Specialists,Write press releases or other media communicat...,Public Relations Specialists,0.854767,0.736280,N/A
